In [2]:
!pip install func-timeout

     |████████████████████████████████| 44 kB 64.5 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for func-timeout: filename=func_timeout-4.3.5-py3-none-any.whl size=15098 sha256=a12ec671348ca818c11ec3d2f6684f9d6f3bf30f7b829e4309d54e5162d9dab9
  Stored in directory: /tmp/pip-ephem-wheel-cache-1tg5gwp_/wheels/a8/92/ca/5bbab358275e310af23b73fc32ebf37d6a7a08c87c8d2cdbc1
Successfully built func-timeout
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
from game_simulation import GameSimulation, SnapshotData
from agent import MoveCommand, ShootCommand, Command
import numpy as np
import typing
import shooting
import evation

def down_ai(gamestate: SnapshotData, my_data: typing.Dict) -> Command:
    '''
    This function is the deliverable that the participants need to upload at the end of the event.
    
    It has to return either a MoveCommand((dx, dy, 0.0)), where dx and dy are float value. 
    (The vector is normalized to 1).
    
    Or it can return a ShootCommand(angle), where the angle is in degrees and in counterclockwise order.
    (Angle 0.0 corresponds to EAST, 270.0° to SOUTH etc...)
    
    The gamestate variable is a collection of information containing the player data as well as the projectile data.
    Please check the SnapshotData class in the agent.py script.
    
    The my_data variable is a dictionnary that persists each time that the function is called. Its an utilitary
    that can be used by the participants at will. 
    
    Its also returned at the END of the game simulation, so it can be used for everything.
    
    The obstactles in game are always the same and described in another document.  Feel free to hardcode them inside the function.
    '''
    if my_data.get('counter') is None:
        my_data['counter'] = 1
    else:
        my_data['counter'] += 1
        
    if gamestate.controlled_player.position[0] > 0:
        return MoveCommand((-1.0, 0.0, 0.0))
    elif gamestate.controlled_player.position[1] < -100:
        return MoveCommand((0.0, 1.0, 0.0))
    
    res = shooting.shoot_on_ennemy_prevision(gamestate)
    
    return ShootCommand(res)

def up_ai(gamestate: SnapshotData, my_data: typing.Dict) -> Command:
    '''
    This function is the deliverable that the participants need to upload at the end of the event.
    
    It has to return either a MoveCommand((dx, dy, 0.0)), where dx and dy are float value. 
    (The vector is normalized to 1).
    
    Or it can return a ShootCommand(angle), where the angle is in degrees and in counterclockwise order.
    (Angle 0.0 corresponds to EAST, 270.0° to SOUTH etc...)
    
    The gamestate variable is a collection of information containing the player data as well as the projectile data.
    Please check the SnapshotData class in the agent.py script.
    
    The my_data variable is a dictionnary that persists each time that the function is called. Its an utilitary
    that can be used by the participants at will. 
    
    Its also returned at the END of the game simulation, so it can be used for everything.
    
    The obstactles in game are always the same and described in another document.  Feel free to hardcode them inside the function.
    '''
    if(gamestate.controlled_player.health<100):
        print("health: ", gamestate.controlled_player.health)
        print("pos", gamestate.controlled_player.position)
    if gamestate.controlled_player.position[0] < 0:
        speed = np.array([1.0, 0.0, 0.0])
        speed = evation.evade_all(None,gamestate,speed)
        return MoveCommand((speed[0],speed[1],speed[2]))
    else:
        speed = -np.array(gamestate.controlled_player.position)
        speed = evation.evade_all(None,gamestate,speed)
        return MoveCommand((speed[0],speed[1],speed[2]))
        
    return ShootCommand(0.0)

In [2]:
import json

# Fetches the installation path of the JAVA virtual machine
with open('jre_path.json', 'r') as file:
    path_dict = json.load(file)

jre_path = path_dict['path']

In [ ]:
simulation_list = []
used_port = 2049

for _ in range(1):
    # GameSimulation takes care of launching a game and running both agents.
    # As GameSimulation is using multiprocessing, many games can be simulated at the same time.
    # However, make sure that each simulation uses a different PORT (for example, start at 2049 and increment
    # by one each time you are running a new simuation).
    simulation = GameSimulation(jre_path, game_time=600.0, ai_time=150.0, commands_per_second=4, port=used_port)
    used_port += 1
    first_agent_data = {}
    second_agent_data = {}

    # Setting the agent username, their AI function as well as their persistent data
    simulation.set_first_agent('agent_0', up_ai, first_agent_data)
    simulation.set_second_agent('agent_1', down_ai, second_agent_data)

    simulation_list.append(simulation)
    
    # Starts the game
    simulation.start_round()

for simulation in simulation_list:
    
    # Waits for the game to end and fetches the results, as well as the persistent data
    results, data = simulation.end_round()

    for agent_result in results:
        print(f'Agent result: {agent_result}')

    for agent_data in data:
        print(f'Agent data: {agent_data}')

Components and Systems are initialized.
Waiting for an agent to connect on port 2049.
Client accepted with address: /127.0.0.1
Initialized python client with time: 150.0, - 1500000026624
Agent connected with username: agent_0 and team: 0. Assigned to player entity: 14
Waiting for an agent to connect on port 2049.
health:  53.47026
pos (-5.841258, 62.672146, 0.0)
health:  53.47026
pos (31.658749, 62.672146, 0.0)
health:  53.47026
pos (54.01095, 32.56186, 0.0)
health:  53.47026
pos (21.895784, 13.20042, 0.0)
health:  53.47026
pos (-10.219396, -6.1610165, 0.0)
health:  8.020466
pos (4.883604, -1.9231282, 0.0)
health:  8.020466
pos (-30.008453, 11.817114, 0.0)
health:  -27.045933
pos (-1000.0, -1000.0, 0.0)
health:  -27.045933
pos (-1000.0, -1000.0, 0.0)
health:  -27.045933
pos (-1000.0, -1000.0, 0.0)
health:  -27.045933
pos (-1000.0, -1000.0, 0.0)
health:  -27.045933
pos (-1000.0, -1000.0, 0.0)
health:  -27.045933
pos (-1000.0, -1000.0, 0.0)
health:  -27.045933
pos (-1000.0, -1000.0, 0.0)